In [1]:
import torch
from torch import nn

In [2]:
of_basic_features = torch.load('kilian/basic_features.pt', map_location='cpu')

extra_msa_feat = torch.load('kilian/extra_msa_feat.pt', map_location='cpu')
msa_feat = of_basic_features['msa_feat'][..., 0]
residue_index = of_basic_features['residue_index'][..., 0]
target_feat = of_basic_features['target_feat'][..., 0]

c_z = 128
c_m  = 256
tf_dim = 22
openfold_param_path = ('openfold/resources/openfold_params/finetuning_ptm_2.pt')
openfold_weights = torch.load(openfold_param_path, map_location='cpu')

batch = {'extra_msa_feat': extra_msa_feat, 'msa_feat': msa_feat, 'residue_index': residue_index, 'target_feat': target_feat}

In [9]:
input_names = [a for a in openfold_weights.keys() if 'input' in a]
print(input_names)

['input_embedder.linear_tf_z_i.weight', 'input_embedder.linear_tf_z_i.bias', 'input_embedder.linear_tf_z_j.weight', 'input_embedder.linear_tf_z_j.bias', 'input_embedder.linear_tf_m.weight', 'input_embedder.linear_tf_m.bias', 'input_embedder.linear_msa_m.weight', 'input_embedder.linear_msa_m.bias', 'input_embedder.linear_relpos.weight', 'input_embedder.linear_relpos.bias']


In [60]:
class Model(nn.Module):
    def __init__(self, c_z, c_m, tf_dim):
        super().__init__()
        self.input_embedder = InputEmbedder(c_z, c_m, tf_dim)

In [61]:
model = Model(c_z, c_m, tf_dim)
input_embedder = model.input_embedder
model.load_state_dict(openfold_weights, strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['recycling_embedder.linear.weight', 'recycling_embedder.linear.bias', 'recycling_embedder.layer_norm_m.weight', 'recycling_embedder.layer_norm_m.bias', 'recycling_embedder.layer_norm_z.weight', 'recycling_embedder.layer_norm_z.bias', 'template_angle_embedder.linear_1.weight', 'template_angle_embedder.linear_1.bias', 'template_angle_embedder.linear_2.weight', 'template_angle_embedder.linear_2.bias', 'template_pair_embedder.linear.weight', 'template_pair_embedder.linear.bias', 'template_pair_stack.blocks.0.tri_att_start.layer_norm.weight', 'template_pair_stack.blocks.0.tri_att_start.layer_norm.bias', 'template_pair_stack.blocks.0.tri_att_start.linear.weight', 'template_pair_stack.blocks.0.tri_att_start.mha.linear_q.weight', 'template_pair_stack.blocks.0.tri_att_start.mha.linear_k.weight', 'template_pair_stack.blocks.0.tri_att_start.mha.linear_v.weight', 'template_pair_stack.blocks.0.tri_att_start.mha.linear_o.weight', 'template_pair_sta

In [62]:
m, z = input_embedder(batch)